In [ ]:
%maven info.picocli:picocli:4.5.2
    
import picocli.CommandLine;
import java.util.concurrent.Callable;
import java.lang.reflect.*;
import java.lang.annotation.*;

<center>
    
# Jak stworzyć swoją pierwszą adnotację w Javie?

### Paweł Bogdan

## Alten

#### 04.02.2021
    
</center>

# Agenda


1. Zdefiniowanie adnotacji
2. Prezentacja adnotacji dostępnych w bibliotece standardowej Java
3. Jak stworzyć prostą adnotację?
3. Prezentacja adnotacji udostępnianych przez inne frameworki
4. Krótka powtórka z refleksji
5. Dlaczego Java nie zawsze widzi adnotację i jak można to zmienić?
6. Co jeszcze można zdefiniować w swojej własnej adnotacji?
7. Zdefiniowanie własnego serializera obiektów do JSONa


<center>

# Definicja adnotacji

</center>

> Annotations, a form of metadata, provide data about a program that is not part of the program itself. Annotations have no direct effect on the operation of the code they annotate.

[Źródło](https://docs.oracle.com/javase/tutorial/java/annotations/index.html)

Służą do

- przekazywania danych kompilatorowi
- zapisania dodatkowych informacji, które są wykorzystywane przez wirtualną maszynę Javy w czasie przetwarzania kodu binarnego lub w czasie działania programu


<center>

# Adnotacje z biblioteki standardowej Javy
    
</center>

<center>

### `@Override`

</center>

In [ ]:
class ExampleOverride {
    @Override
    public String tostring() {
        return "example";
    }
}

<center>
    
### `@FunctionalInterface`

</center>


In [ ]:
@FunctionalInterface
public interface Function {
    double val(double x);
    default void foo() {};
}

<center>

### `@Deprecated`

</center>

In [ ]:
@Deprecated
public class Library {
    public double areaOfCircle(double radius) {
        return Math.PI * radius * radius;
    }
}

<center>

### `@SuppressWarnings`

</center>

```java
public class Main {
    List list = new ArrayList();
    
    @SuppressWarnings("unchecked")
    public void add(int element) {
        list.add(element);
    }
}
```

<center>

# Nasza pierwsza adnotacja
    
</center>

In [ ]:
@interface SimpleAnnotation {}

@SimpleAnnotation
public class MyClass {
    
    @SimpleAnnotation
    public int field;
    
    @SimpleAnnotation
    public void myMethod() {
        System.out.println("FOO");
    }
}

<center>
    
### Można dodać atrybuty

</center>

In [ ]:
@interface SimpleAnnotation {
    String attribute();
}

<center>
    
### Ale wtedy się nie skompiluje...
    
</center>

In [ ]:
@SimpleAnnotation
public class MyClass {
    
    @SimpleAnnotation
    public int field;
    
    @SimpleAnnotation
    public void myMethod() {
        System.out.println("FOO");
    }
}

<center>
    
### Rozwiązanie 1 - dodanie atrybutów
    
</center>

In [ ]:
@interface SimpleAnnotation {
    String attribute();
}

@SimpleAnnotation
public class MyClass {
    
    @SimpleAnnotation
    public int field;
    
    @SimpleAnnotation
    public void myMethod() {
        System.out.println("FOO");
    }
}

<center>
    
### Rozwiązanie 2 - wartość domyślna
    
</center>

In [ ]:
@interface SimpleAnnotation {
    String attribute();
}

@SimpleAnnotation
public class MyClass {
    
    @SimpleAnnotation
    public int field;
    
    @SimpleAnnotation
    public void myMethod() {
        System.out.println("FOO");
    }
}

<center>
    
### A można bez nazwy atrybutu?
    
</center>

In [ ]:
@interface SimpleAnnotation {
}

@SimpleAnnotation("Hello!")
public class MyClass {
    
    @SimpleAnnotation
    public int field;
    
    @SimpleAnnotation
    public void myMethod() {
        System.out.println("FOO");
    }
}

<center>
    
### Jakiego typu może być atrybut?
    
</center>

<center>
    
### Dowolny typ prymitywny
    
</center>

In [ ]:
@interface SimpleAnnotation {
    int value() default 0;
    double attribute();
}

<center>
    
### Obiekt klasy String
    
</center>

In [ ]:
@interface SimpleAnnotation {
    String value() default 0;
}

<center>
    
### Enum
    
</center>

In [ ]:
enum Side {
    RIGHT, LEFT;
}

@interface SimpleAnnotation {
    Side value() default RIGHT;
}

<center>
    
### Obiekt klasy `Class`
    
</center>

In [ ]:
@interface SimpleAnnotation {
    Class<?> value();
}

<center>
    
### Tablica powyższych
    
</center>

In [ ]:
@interface SimpleAnnotation {
    String[] value();
}

@SimpleAnnotation({"a", "b", "c"})
class SimpleClass{}

<center>

# Adnotacje w innych bibliotekach
    
</center>

<center>


## Spring

## JAXB

## Lombock

## ...
    
</center>

<center>

# Biblioteka `picocli`
    
</center>

In [ ]:
@CommandLine.Command(
    name="example",
    mixinStandardHelpOptions=false,
    description="Just example command line app"
)
class ExampleCommandLineApp implements Callable<Integer> {
    @CommandLine.Option(names="-n", description="name of person to greet")
    String name;
    
    @Override 
    public Integer call() {
        System.out.println("Witaj " + name + "!");
        return 0;
    }
}

int status = new CommandLine(new ExampleCommandLineApp()).execute(new String[]{"nothing"});

<center>

### Są jeszcze inne atrybuty
    
</center>

In [ ]:
@CommandLine.Command(
    name="example",
    mixinStandardHelpOptions=true,
    description="Just example command line app"
)
class ExampleCommandLineApp implements Callable<Integer> {
    @CommandLine.Option(
        names={"-n", "--name"}, 
        description="name of person to greet",
        paramLabel="NAME",
        required=false,
        defaultValue="Pawel"
    )
    String name;
    
    @Override 
    public Integer call() {
        System.out.println("Witaj " + name + "!");
        return 0;
    }
}

int status = new CommandLine(new ExampleCommandLineApp()).execute(new String[]{"-n", "Bogdan"});

<center>

### I inne adnotacje
    
</center>

In [ ]:
@CommandLine.Command(name="example", mixinStandardHelpOptions=true, description="Just example command line app")
class ExampleCommandLineApp implements Callable<Integer> {
    @CommandLine.Option(names={"-n", "--name"}, description="name of person to greet", defaultValue="Pawel")
    String name;
    @CommandLine.Parameters(index="0", paramLabel="PATH", description="Path to file", defaultValue="/etc/passwd" )
    String param;
    @Override 
    public Integer call() {
        System.out.println("Witaj " + name + "!");
        System.out.println("Chcesz przeczytać plik: " + param);
        return 0;
    }
}

int status = new CommandLine(new ExampleCommandLineApp()).execute(new String[]{"-n", "pawel"});

<center>

# Krótka powtórka z refleksji
    
</center>

In [ ]:
Class<?> c = String.class;

Field[] fields = c.getDeclaredFields();

for (Field f : fields) {
    System.out.println(f.getName();
}


<center>
I teraz możemy
</center>

In [ ]:
String s = "pawel";
Field f = c.getDeclaredField("value");
f.setAccessible(true);
Object o = f.get(s);

System.out.println(s);

<center>
Podobnie możemy z adnotacjami
</center>

In [ ]:
@interface SimpleAnnotation{}

class SimpleClass {
    @Override
    @SimpleAnnotation
    @CommandLine.Command
    public String toString() {
        return "hahah";
    }
}

Method m = SimpleClass.class.getMethod("toString");
for (Annotation a : m.getAnnotations()) {
    System.out.println(a.annotationType().getSimpleName());
}

<center>
    
### A teraz będzie jeszcze dziwniej...
    
</center>

Gdy skompilujemy klasę `SimpleClass` i następnie ją zdeasemblujemy poleceniem: 
```bash
 javap -v -p SimpleClass.class
```
To dostaniemy:
```
    RuntimeVisibleAnnotations:
      0: #15()
        picocli.CommandLine$Command
    RuntimeInvisibleAnnotations:
      0: #17()
        SimpleAnnotation
```

<center>
    
# Dlaczego Java nie zawsze widzi adnotację?
# Jak można to zmienić?
    
</center>

<center>

## Odpowiedź jest całkiem prosta
    
</center>

Dla każdej adnotacji możemy ustawić jeden z trzech poziomów widoczności:

1. widoczna w czasie kompilacji
2. widoczna w kodzie bajtowym
3. widoczna w trakcie uruchomienia aplikacji

### Domyślnym poziomem widoczności jest poziom 2

<center>

## Jak zdefiniować inny poziom widoczności?
    
</center>

<center>

## Przez położenie adnotacji na definicji adnotacji
    
</center>

In [ ]:
@Retention(RetentionPolicy.SOURCE)
@interface SimpleAnnotation {
    String value() default "";
}

Gdzie adnotacja jest zdefiniowana:

```java
@Documented
@Retention(RetentionPolicy.RUNTIME)
@Target({ElementType.ANNOTATION_TYPE})
public @interface Retention {
    RetentionPolicy value() default RetentionPolicy.CLASS;
}
```

A ten enum, użyty w definicji adnotacji jest zdefiniowany:

```java
public enum RetentionPolicy {
    SOURCE,
    CLASS,
    RUNTIME;
}
```

<center>

## Sprawdźmy jak to działa
    
</center>

In [ ]:
@Retention(RetentionPolicy.RUNTIME)
@interface SimpleAnnotation{}

class SimpleClass {
    @Override
    @SimpleAnnotation
    @CommandLine.Command
    public String toString() {
        return "hahah";
    }
}

Method m = SimpleClass.class.getMethod("toString");
for (Annotation a : m.getAnnotations()) {
    System.out.println(a.annotationType().getSimpleName());
}

<center>
    
# Co jeszcze można zdefiniować w swojej własnej adnotacji?    
    
</center>

<center>

### Ograniczyć, na co adnotację możemy położyć    
    
</center>


```java
@Documented
@Retention(RetentionPolicy.RUNTIME)
@Target({ElementType.ANNOTATION_TYPE})
public @interface Target {
    ElementType[] value();
}
```

In [ ]:
@Target(ElementType.TYPE)
@Retention(RetentionPolicy.RUNTIME)
@interface SimpleAnnotation{}

class SimpleClass {
    @SimpleAnnotation
    int field;
}

<center>

### Pozwolić na dziedziczenie 
    
</center>


```java
@Documented
@Retention(RetentionPolicy.RUNTIME)
@Target({ElementType.ANNOTATION_TYPE})
public @interface Inherited {
}
```

<center>

### Pozwolić na wielokrotne położenie adnotacji
    
</center>

```java
@Documented
@Retention(RetentionPolicy.RUNTIME)
@Target(ElementType.ANNOTATION_TYPE)
public @interface Repeatable {
    Class<? extends Annotation> value();
}
```

In [ ]:
@Retention(RetentionPolicy.RUNTIME)
@interface SimpleAnnotations {
    SimpleAnnotation[] value();
}

@Repeatable(SimpleAnnotations.class)
@Retention(RetentionPolicy.RUNTIME)
@interface SimpleAnnotation {
    String value() default "";
}

@SimpleAnnotation()
@SimpleAnnotation()
public class Foo {
    void bar() {}
}

Class<Foo> clazz = Foo.class;
for (Annotation a : clazz.getAnnotations()) {
    System.out.println(a.annotationType().getSimpleName());
}

<center>
    
# Zdefiniowanie własnego serializera obiektów do JSONa   
    
</center>

In [ ]:
@Retention(RetentionPolicy.RUNTIME)
@Target(ElementType.TYPE)
public @interface JSONObject {
}

@Retention(RetentionPolicy.RUNTIME)
@Target(ElementType.FIELD)
public @interface JSONField {
    String value() default "";
}

In [ ]:
@JSONObject
public class Student {
    @JSONField("Imie")
    public String firstName;
    @JSONField("Nazwisko")
    public String lastName;
    public String address;
    @JSONField
    public int age;
    @JSONField
    public double height;
    public Student(String firstName, String lastName, String address, int age, double height) {
        this.firstName = firstName;
        this.lastName = lastName;
        this.address = address;
        this.age = age;
        this.height = height;
    }
}

In [ ]:
public static String serializeToJSON(Object o) throws IllegalAccessException {
    Class<?> clazz = o.getClass();
    if (!clazz.isAnnotationPresent(JSONObject.class)) {
        throw new IllegalArgumentException("Not supported class");
    }
    Field[] fields = clazz.getDeclaredFields();
    StringBuilder result = new StringBuilder("{");
    result.append(System.lineSeparator());
    for (Field f : fields) {
        
    }
    result.append("}");
    return result.toString();
}

Student s = new Student("Doktor", "Ziel", "Zielony Backlog 13", 44, 1.56);
System.out.println(serializeToJSON(s));

<center>
    
# Podsumowanie  
    
</center>

1. Zobaczyliśmy adnotacje z biblioteki standardowej
2. Nauczyliśmy się tworzyć własne adnotacje
3. Poznaliśmy bibliotekę `picocli`
4. Poznaliśmy poziomy widoczności adnotacji
5. Dowiedzieliśmy się, że właściwości adnotacji dodaje się przez adnotację
6. Napisaliśmy własne adnotacje i użyliśmy ich do rozwiązania rzeczywistego problemu

<center>
    
# No ale co z tym Lombokiem?
    
</center>

Sami [twórcy opisują](https://projectlombok.org/) to jako:

> Project Lombok is a java library that automatically plugs into your editor and build tools, spicing up your java.
Never write another getter or equals method again, with one annotation your class has a fully featured builder, Automate your logging variables, and much more. 

### Wady

- Łamie enkapsulację
- Dodaje magię do kodu
- Utrudnia debugowanie


### Zalety

- Przyspiesza pracę nad projektem

<center>
    
# Dziękuję za uwagę

    
### znajdziecie mnie na https://zielony-backlog.pl
    
    
</center>


